<a href="https://colab.research.google.com/github/sahil-ansari-15/Bank-Customer-Churn-Prediction-using-ANN-Deep_Learning/blob/main/ANN_Hyperparameter_tuning.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Artificial Neural Network

### Data Preprocessing

In [1]:
# Importing the libraries

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [2]:
# Importing the dataset

df_dataset = pd.read_csv('Churn_Modelling.csv')
X = df_dataset.iloc[:, 3:13]
y = df_dataset.iloc[:, 13]

In [3]:
df_dataset.head()

,RowNumber,CustomerId,Surname,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Exited
0,1,15634602,Hargrave,619,France,Female,42,2,0.00,1,1,1,101348.88,1
1,2,15647311,Hill,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0
2,3,15619304,Onio,502,France,Female,42,8,159660.80,3,1,0,113931.57,1
3,4,15701354,Boni,699,France,Female,39,1,0.00,2,0,0,93826.63,0
4,5,15737888,Mitchell,850,Spain,Female,43,2,125510.82,1,1,1,79084.10,0


In [4]:
df_dataset.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10000 entries, 0 to 9999
Data columns (total 14 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   RowNumber        10000 non-null  int64  
 1   CustomerId       10000 non-null  int64  
 2   Surname          10000 non-null  object 
 3   CreditScore      10000 non-null  int64  
 4   Geography        10000 non-null  object 
 5   Gender           10000 non-null  object 
 6   Age              10000 non-null  int64  
 7   Tenure           10000 non-null  int64  
 8   Balance          10000 non-null  float64
 9   NumOfProducts    10000 non-null  int64  
 10  HasCrCard        10000 non-null  int64  
 11  IsActiveMember   10000 non-null  int64  
 12  EstimatedSalary  10000 non-null  float64
 13  Exited           10000 non-null  int64  
dtypes: float64(2), int64(9), object(3)
memory usage: 1.1+ MB


In [5]:
#Create dummy variables for One-hot encoding

geography = pd.get_dummies(X["Geography"],drop_first=True)
gender = pd.get_dummies(X['Gender'],drop_first=True)

In [6]:
df_dataset.Geography.value_counts()

France     5014
Germany    2509
Spain      2477
Name: Geography, dtype: int64

In [7]:
geography.head()

,Germany,Spain
0,0,0
1,0,1
2,0,0
3,0,0
4,0,1


In [8]:
# Concatenate the Data Frames

X = pd.concat([X,geography,gender],axis=1)

In [9]:
X.head(2)

,CreditScore,Geography,Gender,Age,Tenure,Balance,NumOfProducts,HasCrCard,IsActiveMember,EstimatedSalary,Germany,Spain,Male
0,619,France,Female,42,2,0.00,1,1,1,101348.88,0,0,0
1,608,Spain,Female,41,1,83807.86,1,0,1,112542.58,0,1,0


In [10]:
# Drop Unnecessary columns

X = X.drop(['Geography','Gender'],axis=1)

In [11]:
# Splitting the dataset into the Training set and Test set

from sklearn.model_selection import train_test_split

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.2, random_state = 0)

In [12]:
# Feature Scaling

from sklearn.preprocessing import StandardScaler
sc = StandardScaler()

X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

### Building ANN model with ***Hyperparameters tuning***

In [13]:
#!pip install keras-tuner

In [14]:
# Importing the Keras libraries and packages
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LeakyReLU,PReLU,ELU
from keras.layers import Dropout

from kerastuner.tuners import RandomSearch

<ipython-input-14-2d10b49fc61c>:8: DeprecationWarning: `import kerastuner` is deprecated, please use `import keras_tuner`.
  from kerastuner.tuners import RandomSearch


Hyperparameters

1. How many number of hidden layers we should have?
2. How many number of neurons we should have in hidden layers?
3. Learning Rate

In [21]:
def build_model(hp):
    model = Sequential()
    for i in range(hp.Int('num_layers', 2, 20)):
        model.add(Dense(units=hp.Int('units_' + str(i),
                                            min_value=32,
                                            max_value=512,
                                            step=32),
                               activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(
        optimizer=keras.optimizers.Adam(
            hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])),
        loss='binary_crossentropy',
        metrics=['binary_accuracy'])
    return model

In [22]:
tuner = RandomSearch(
    build_model,
    objective='val_binary_accuracy',
    max_trials=5,
    executions_per_trial=3,
    directory='project',
    project_name='customer_churn')

In [23]:
tuner.search_space_summary()

Search space summary
Default search space size: 22
num_layers (Int)
{'default': None, 'conditions': [], 'min_value': 2, 'max_value': 20, 'step': 1, 'sampling': 'linear'}
units_0 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_1 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
learning_rate (Choice)
{'default': 0.01, 'conditions': [], 'values': [0.01, 0.001, 0.0001], 'ordered': True}
units_2 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_3 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_4 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_5 (Int)
{'default': None, 'conditions': [], 'min_value': 32, 'max_value': 512, 'step': 32, 'sampling': 'linear'}
units_6

In [24]:
tuner.search(X_train, y_train,
             epochs=5,
             validation_data=(X_test, y_test))

In [25]:
tuner.results_summary()

Results summary
Results in project/customer_churn
Showing 10 best trials
Objective(name="val_binary_accuracy", direction="max")

Trial 2 summary
Hyperparameters:
num_layers: 6
units_0: 288
units_1: 512
learning_rate: 0.001
units_2: 320
units_3: 448
units_4: 512
units_5: 480
units_6: 256
units_7: 320
units_8: 64
units_9: 224
units_10: 96
units_11: 448
units_12: 128
units_13: 416
units_14: 416
units_15: 32
units_16: 192
units_17: 288
units_18: 352
units_19: 288
Score: 0.8519999980926514

Trial 3 summary
Hyperparameters:
num_layers: 2
units_0: 256
units_1: 96
learning_rate: 0.001
units_2: 256
units_3: 32
units_4: 352
units_5: 480
units_6: 384
units_7: 96
units_8: 96
units_9: 96
units_10: 160
units_11: 192
units_12: 96
units_13: 224
units_14: 256
units_15: 512
units_16: 384
units_17: 288
units_18: 160
units_19: 256
Score: 0.8178333441416422

Trial 4 summary
Hyperparameters:
num_layers: 17
units_0: 64
units_1: 96
learning_rate: 0.01
units_2: 32
units_3: 192
units_4: 352
units_5: 480
units_6

In [20]:
aaaaaaaaaaaaaaaaaaaaaaaaaaaa

NameError: ignored

In [ ]:
# Initialising the ANN
classifier = Sequential()

# Adding the input layer and the first hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu',input_dim = 11)) 
### units = output_dim        ### kernel_initializer = init

# Adding the second hidden layer
classifier.add(Dense(units = 6, kernel_initializer = 'he_uniform',activation='relu'))

# Adding the output layer
classifier.add(Dense(units = 1, kernel_initializer = 'glorot_uniform', activation = 'sigmoid'))

# Compiling the ANN
#classifier.compile(optimizer = 'Adam', loss = 'binary_crossentropy', metrics = ['accuracy'])
classifier.compile(optimizer = keras.optimizers.Adam(), loss = 'binary_crossentropy', metrics = ['accuracy'])

classifier.summary()

# Fitting the ANN to the Training set
model_history = classifier.fit(X_train, y_train,validation_split=0.33, batch_size = 50, epochs = 100)

# list all data in history
print(model_history.history.keys())

In [ ]:
# summarize history for accuracy

plt.plot(model_history.history['accuracy'])
plt.plot(model_history.history['val_accuracy'])
plt.title('model accuracy')
plt.ylabel('accuracy')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

In [ ]:
# summarize history for loss
plt.plot(model_history.history['loss'])
plt.plot(model_history.history['val_loss'])
plt.title('model loss')
plt.ylabel('loss')
plt.xlabel('epoch')
plt.legend(['train', 'test'], loc='upper left')
plt.show()

### Making the predictions and evaluating the model

In [ ]:
# Predicting the Test set results
y_pred = classifier.predict(X_test)
y_pred[:5]

In [ ]:
y_pred_transformed = np.where(y_pred > 0.5, 1, 0)
y_pred_transformed[:5]

In [ ]:
y_test[:5]

In [ ]:
type(y_test)

In [ ]:
# Making the Confusion Matrix
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred_transformed)
cm

In [ ]:
# Calculate the Accuracy
from sklearn.metrics import accuracy_score
score=accuracy_score(y_pred_transformed,y_test)
score